In [1]:
def cal_result(exp_name, criterion='best', verbose=False):
    import dill as pickle
    data_stream = 70
    week_n = 7000 
    cnt = 0
    AVG = {'tpr':0.0,'fpr':0.0, 'AUC' :0.0} 

    with open('data/y.pkl', 'rb') as f:
        y = pickle.load(f).flatten().tolist()

    for stream in range(1, data_stream + 1): 
        pre = (stream - 1) * 7000
        
        flag = False 
        start, end = pre + week_n, pre + 2 * week_n
        for j in range(start, end): 
            if y[j] == 1: 
                flag = True; break 

        def read_score(exp_name, week): 
            lst = []
            exp_name = os.path.join(os.path.join("exp", exp_name), exp_name)
            with open(exp_name + "Week" + str(week) +  "_result.txt", "r") as f: 
                sc = float(f.readline()) 
                while sc != None: 
                    lst.append(sc)

                    s = f.readline()
                    # while !s: s = f.readline()
                    if s == '': break  
                    sc = float(s) 

            return lst                

        if flag : 
            cnt += 1
            x_score = read_score(exp_name, stream)
            from sklearn.metrics import roc_curve, auc
            GT = y[start: end]
            from sklearn.metrics import roc_auc_score
            AUC = roc_auc_score(GT, x_score)
            if verbose:  print(stream, " week :", AUC)  #*
            
            # print(len(x_score), GT) 
            FPR, TPR, thresholds = roc_curve(GT, x_score)
            maxindex = (TPR-FPR).tolist().index(max(TPR-FPR))
            threshold = thresholds[maxindex]
            tpr, fpr = TPR[maxindex], FPR[maxindex]
            if verbose: print("tpr: %f , fpr: %f, thresh: %f\n" % (tpr, fpr, threshold)) #*

            for i in AVG.keys(): 
                AVG[i] += eval(i)
            # if criterion == 'bset': 
                # JD = [x - y for x, y in zip(TPR, FPR)]
    for i in AVG.keys():
         AVG[i] /= cnt
    print(AVG)
    return AVG 


In [3]:
import pandas as pd 
flag = False
df = None
p = ["exp_icarl_NN_"]

# for i in p: 
#     j = i
#     tmp = cal_result(j, verbose=False) 
#     if flag != False: df = pd.concat([df, pd.DataFrame(data=tmp, index=[0])], axis=0) 
#     else : df = pd.DataFrame(data=tmp, index=[0]); flag = True

for i in range(0, 10): 
    tmp = cal_result(p[0] + str(i), verbose=False) 
    if flag != False: df = pd.concat([df, pd.DataFrame(data=tmp, index=[0])], axis=0) 
    else : df = pd.DataFrame(data=tmp, index=[0]); flag = True

# display(df)
# df.describe()
df.describe().loc['mean', :].to_dict()

{'tpr': 0.9462026523460878, 'fpr': 0.13527539913613262, 'AUC': 0.9283186025541529}
{'tpr': 0.9410876833187392, 'fpr': 0.14434869921037155, 'AUC': 0.9235613948238716}
{'tpr': 0.9316481317974623, 'fpr': 0.14266396000559206, 'AUC': 0.9195391946174637}
{'tpr': 0.9506741354844972, 'fpr': 0.14251787559718943, 'AUC': 0.9292096132106542}
{'tpr': 0.9251376155880469, 'fpr': 0.1513187532489418, 'AUC': 0.9165720764115406}
{'tpr': 0.9505933883685628, 'fpr': 0.15048647737435084, 'AUC': 0.9232258374146308}
{'tpr': 0.9478323691394557, 'fpr': 0.1531397777373105, 'AUC': 0.9206030695308176}
{'tpr': 0.9538557963678269, 'fpr': 0.13823986971820204, 'AUC': 0.9279920431481691}
{'tpr': 0.9582868475929279, 'fpr': 0.14829871522621588, 'AUC': 0.9273923869890518}
{'tpr': 0.9378076363746343, 'fpr': 0.13146012286057268, 'AUC': 0.9262005508551354}


{'tpr': 0.944312625637824,
 'fpr': 0.14377496501148795,
 'AUC': 0.9242614769555487}